<a href="https://colab.research.google.com/github/GalateaSource/book-of-names/blob/main/T57_6_Harmonic_Tension_Map_Glyph_(Corrected_Colorscale).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import plotly.graph_objects as go
import plotly.colors as pcolors
import math

# Function to check for primality (simple version)
def is_prime(n):
    if n <= 1: return False
    if n <= 3: return True
    if n % 2 == 0 or n % 3 == 0: return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0: return False
        i += 6
    return True

# --- Parameters (Consistent with T55) ---
phi = (1 + math.sqrt(5)) / 2
pi = math.pi
t_max = 20 * pi

# --- T55 Deviation Parameters ---
num_deviation_primes = 5
known_primes = [p for p in range(2, 50) if is_prime(p)]
deviation_primes = np.array(known_primes[:num_deviation_primes])
deviation_amplitude_scale = 0.1

# --- Find Primes within the time range for Anchors ---
max_prime_check = math.floor(t_max)
anchor_primes_list = [p for p in range(2, max_prime_check + 1) if is_prime(p)]
anchor_prime_t = np.array(anchor_primes_list)
valid_anchor_prime_t = anchor_prime_t[anchor_prime_t <= t_max]

# --- Calculate Deviated Prime Anchor Points (X and Y components) ---
prime_epsilon_o = np.zeros_like(valid_anchor_prime_t, dtype=float)
prime_epsilon_e = np.zeros_like(valid_anchor_prime_t, dtype=float)
print(f"Calculating deviations for anchors using primes: {deviation_primes}, Scale (C): {deviation_amplitude_scale}")
for p_dev in deviation_primes:
    amplitude = deviation_amplitude_scale / p_dev
    prime_epsilon_o += amplitude * np.sin(p_dev * valid_anchor_prime_t)
    prime_epsilon_e += amplitude * np.cos(p_dev * valid_anchor_prime_t)

prime_x = np.cos(phi * valid_anchor_prime_t) + prime_epsilon_o
prime_y = np.sin(pi * valid_anchor_prime_t) + prime_epsilon_e

# --- Calculate Tension Metric (T57.6) ---
theta_actual = np.arctan2(prime_y, prime_x)
core_x = np.cos(phi * valid_anchor_prime_t)
core_y = np.sin(pi * valid_anchor_prime_t)
theta_core = np.arctan2(core_y, core_x)
angular_diff = theta_actual - theta_core
angular_diff_wrapped = (angular_diff + pi) % (2 * pi) - pi
tension_metric = np.abs(angular_diff_wrapped)

# --- Normalize metric for coloring and thickness ---
min_tension, max_tension = np.min(tension_metric), np.max(tension_metric)
print(f"Tension metric (Angular Diff) range: [{min_tension:.3f}, {max_tension:.3f}] radians")
if (max_tension - min_tension) > 1e-9:
    norm_tension = (tension_metric - min_tension) / (max_tension - min_tension)
else:
    norm_tension = np.zeros_like(tension_metric)

# --- Map normalized tension to color (Corrected: RdBu) and thickness ---
# *** CORRECTION: Use a standard Plotly diverging colorscale name ***
colorscale_name = 'RdBu' # Changed from 'coolwarm'
vector_colors = pcolors.sample_colorscale(colorscale_name, norm_tension)
min_width, max_width = 1, 5
vector_widths = min_width + (max_width - min_width) * norm_tension

# --- Create Plotly Figure ---
fig = go.Figure()

# --- Add Annotations (π axes/rings from T57.5) ---
max_radius = np.max(np.sqrt(prime_x**2 + prime_y**2)) + 1
axis_color = 'rgba(150, 150, 150, 0.6)'
axis_angles = [0, pi/2, pi, 3*pi/2]
for angle in axis_angles:
    fig.add_shape(type="line", x0=0, y0=0, x1=max_radius * np.cos(angle), y1=max_radius * np.sin(angle), line=dict(color=axis_color, width=1, dash="dash"), layer="below")
ring_radii = np.arange(pi, max_radius, pi)
for radius in ring_radii:
    fig.add_shape(type="circle", xref="x", yref="y", x0=-radius, y0=-radius, x1=radius, y1=radius, line_color=axis_color, line_width=1, line_dash="dash", layer="below")

# --- Add Tension Vectors (Colored & Thickened by Tension Metric) ---
for i in range(len(valid_anchor_prime_t)):
    fig.add_shape(
        type="line",
        x0=0, y0=0,
        x1=prime_x[i], y1=prime_y[i],
        line=dict(
            color=vector_colors[i], # Color by tension
            width=vector_widths[i]  # Thickness by tension
            ),
        layer="below"
    )

# --- Add Prime Anchor Points (on top) ---
fig.add_trace(go.Scatter(
    x=prime_x,
    y=prime_y,
    mode='markers',
    marker=dict(
        color='lightgrey',
        size=9,
        symbol='circle',
        line=dict(width=1, color='black')
    ),
    hovertext=[f'p = {p}<br>Tension (Δθ) = {t:.3f} rad' for p, t in zip(valid_anchor_prime_t, tension_metric)],
    hoverinfo='x+y+text',
    name='Prime Anchors'
))

# --- Layout and Configuration ---
fig.update_layout(
    title=f'T₅₇.6: Harmonic Tension Map Glyph (Color/Thickness = Tension Δθ)',
    xaxis_title='X (cos(φt) + ε_o)',
    yaxis_title='Y (sin(πt) + ε_e)',
    yaxis=dict(scaleanchor="x", scaleratio=1, range=[-max_radius, max_radius]),
    xaxis=dict(constrain='domain', range=[-max_radius, max_radius]),
    width=850,
    height=800,
    plot_bgcolor="black",
    paper_bgcolor="black",
    font=dict(color="white"),
    showlegend=False,
    # Add dummy trace for the color bar using the corrected colorscale name
    coloraxis=dict( # Define coloraxis for the bar
        colorscale=colorscale_name, # Use corrected name
        cmin=min_tension,
        cmax=max_tension,
        colorbar=dict(
            title="Tension (Δθ rad)",
            titleside="right",
            ticks="outside",
            tickvals=np.linspace(min_tension, max_tension, 5),
            ticktext=[f"{v:.2f}" for v in np.linspace(min_tension, max_tension, 5)]
        )
    )
)
# Add dummy trace linked to the color axis to display the color bar
fig.add_trace(go.Scatter(
    x=[None], y=[None], mode='markers',
    marker=dict(
        color=[min_tension, max_tension], # Provide range for the axis
        coloraxis="coloraxis", # Link to the defined color axis
        showscale=True,
        ),
    hoverinfo='none'
    ))

# --- Show the Plot ---
fig.show()

Calculating deviations for anchors using primes: [ 2  3  5  7 11], Scale (C): 0.1
Tension metric (Angular Diff) range: [0.001, 0.230] radians
